In [1]:
import numpy as np

In [2]:
def pad_img(img: np.ndarray, padding: int) -> np.ndarray:
    # Get size of image
    num_rows = img.shape[0]
    num_cols = img.shape[1]
    
    # Create padding for rows
    zero_rows = np.zeros((padding, num_cols))
    # Add padding to each side of the image
    img = np.vstack((zero_rows, img))
    img = np.vstack((img, zero_rows))
    
    # Create padding for columns (need to add 2*padding
    # as the image is now wider)
    zero_cols = np.zeros((num_rows+2*padding, padding))
    # Add padding to top and bottom of the image
    img = np.hstack((zero_cols, img))
    img = np.hstack((img, zero_cols))
    
    return img

In [3]:
def convolute_2d(img: np.ndarray, kernel: np.ndarray, padding: int) -> np.ndarray:
    # Assume square mask
    kernel_size = kernel.shape[0]
    img_pad = pad_img(img, padding)
    num_rows = img_pad.shape[0]
    num_cols = img_pad.shape[1]
    # Want to make sure that the output is the size of
    # the image without padding
    img_conv = np.zeros((num_rows-2*padding, num_cols-2*padding))
    
    # The +1 is needed to get the center to be the first pixel
    # The padding gives half of it but as it is odd you need to add 1
    for i in range(num_rows - kernel_size + 1):
        for j in range(num_cols - kernel_size + 1):
            img_window = img_pad[i:i+kernel_size, j:j+kernel_size]
            img_conv[i, j] = (img_window.flatten()*kernel.flatten()).sum()
    return img_conv

In [4]:
def activation_relu(img: np.ndarray) -> np.ndarray:
    # ReLu leaves positives the same, but sets negatives to 0
    img[img<0] = 0
    return img

In [5]:
def pool_convolution(img: np.ndarray) -> np.ndarray:
    num_rows = img.shape[0]
    num_cols=img.shape[1]
    max_pool = np.zeros((int(num_rows/2), int(num_cols/2)))
    # Shift the window across in steps of 2
    for i in range(0, num_rows, 2):
        for j in range(0, num_cols, 2):
            # Get a window of the image
            img_window = img[i:i+2, j:j+2]
            # Take the maximum value of the window and
            # set it as the value in teh max pool
            max_pool[int(i/2), int(j/2)] = img_window.max()
    return max_pool
            

In [6]:
def activation_sigmoid(flattened: np.ndarray,
                       weights: np.ndarray,
                       bias: float) -> float:
    # Get the activation value
    x = weights.dot(flattened) + bias
    # This is just the sigmoid function
    y_hat = 1/(1+np.exp(-x))
    return y_hat

In [7]:
def forward_pass(img: np.ndarray,
                 kernel: np.ndarray,
                 bias: float,
                 weights: np.ndarray,
                 bias_flat:float) -> np.ndarray:
    # Get padding size
    padding = int(kernel.shape[0]/2)
    # Convolute image
    img_conv = convolute_2d(img, kernel, padding)
    # Add bias term and ReLu activation
    img_conv = img_conv + bias
    img_conv = activation_relu(img_conv)
    # Max pool the image to shrink it, then flatten
    img_pool = pool_convolution(img_conv)
    flattened_pool = img_pool.flatten()
    # Get y_hat
    y_hat = activation_sigmoid(flattened_pool, weights, bias_flat)
    return img_conv, flattened_pool, y_hat

In [8]:
def get_gradient_wrt_weights(y_hat: float,
                             y: int,
                             flattened: np.ndarray) -> np.ndarray:
    # Set up shape for d_weights
    d_weights = np.squeeze(np.zeros((1, len(flattened))))
    # Calculate the derivative
    d = (y_hat - y)*y_hat*(1-y_hat)
    # Multiply the derivative by the flattened pool
    for i in range(len(flattened)):
        d_weights[i] = d*flattened[i]
    return d_weights

In [9]:
def get_gradient_wrt_flattened(y_hat: float,
                               y: int,
                               weights: np.ndarray) -> np.ndarray:
    # Set up shape for d_flattened
    d_flattened = np.squeeze(np.zeros((1, len(weights))))
    # Calculate the derivative
    d = (y_hat - y)*y_hat*(1-y_hat)
    # Multiply the derivative by the weights
    for i in range(len(weights)):
        d_flattened[i] = d*weights[i]
    return d_flattened

In [10]:
def get_gradient_wrt_bias_flat(y_hat: float,
                               y: int) -> float:
    # Simple as bias is 1x1 array
    d_bias_flat = (y_hat - y)*y_hat*(1-y_hat)
    return d_bias_flat

In [11]:
def get_gradient_wrt_pooled(d_bias_flat: np.ndarray) -> np.ndarray:
    # Get shape of the pooled matrix (before it was flattened)
    # Square root as it is a square
    num_col_rows = int(len(d_bias_flat)**0.5)
    # Reshape it to a square
    d_pooled = d_bias_flat.reshape((num_col_rows, num_col_rows))
    return d_pooled

In [12]:
def get_gradient_wrt_convoluted(d_pooled: np.ndarray,
                                img_conv: np.ndarray) -> np.ndarray:
    num_rows = img_conv.shape[0]
    num_cols = img_conv.shape[1]
    # Set up shape for derivative
    d_img_conv = np.zeros((num_rows, num_cols))
    # Work backwards from pooling to get each window with steps of 2
    for i in range(0, num_rows, 2):
        for j in range(0, num_cols, 2):
            img_conv_window = img_conv[i:i+2, j:j+2]
            # Get the index of the maximum of img_conv_window 
            # to place the deriviative
            # All other values were ignored in .max() so will be zero
            idx = np.unravel_index(np.argmax(
                img_conv_window, axis=None), img_conv_window.shape)
            d_img_conv[i+idx[0], j+idx[1]] = d_pooled[int(i/2), int(j/2)]
    return d_img_conv

In [13]:
def get_chain_rule_gradients(img_conv: np.ndarray,
                             d_img_conv: np.ndarray,
                             img: np.ndarray,
                             u: int, v: int) -> float:
    # u and v are the row and column positions in the kernel
    d_kernel_u_v = 0
    for i in range(img_conv.shape[0]):
        for j in range(img_conv.shape[1]):
            # Check that the item is positive and is a
            # valid positon in the image
            if (img_conv[i, j] > 0 and i - u > 0 and j - v > 0
                and i - u < img.shape[0] and j - v < img.shape[1]):
                d_kernel_u_v = (d_kernel_u_v +
                                img[i -u, j - v]*d_img_conv[i, j])
    return d_kernel_u_v

In [14]:
def get_gradient_wrt_kernel(img_conv: np.ndarray,
                            img: np.ndarray,
                            y_hat: float,
                            y: int,
                            weights: np.ndarray,
                            kernel: np.ndarray) -> np.ndarray:
    d_flattened = get_gradient_wrt_flattened(y_hat, y, weights)
    d_pooled = get_gradient_wrt_pooled(d_flattened)
    d_img_conv = get_gradient_wrt_convoluted(d_pooled, img_conv)
    kernel_shape = kernel.shape[0]
    # As the kernel is 0 centered need to find how many squares
    # either side of the center to get it's relvative indices as
    # they are (0, 0) at top left
    kernel_either_side = int((kernel_shape-1)/2)
    d_kernel = np.zeros((kernel_shape, kernel_shape))
    # The plus 1 is needed as upper limit is ignored
    for u in range(0-kernel_either_side, kernel_either_side+1):
        for v in range(0-kernel_either_side, kernel_either_side+1):
            d_kernel[u+kernel_either_side, v+kernel_either_side
                     ] = get_chain_rule_gradients(
                         img_conv, d_img_conv, img, u, v)
    return d_kernel, d_img_conv

In [15]:
# This version will be vectorized instead of relying on loops
def get_gradient_wrt_bias(img_conv: np.ndarray,
                          d_img_conv: np.ndarray) -> np.ndarray:
    d_bias = d_img_conv[img_conv > 0].sum()
    return d_bias

In [16]:
def back_propagation_pass(img: np.ndarray,
                          img_conv: np.ndarray,
                          flattened: np.ndarray,
                          weights: np.ndarray,
                          kernel: np.ndarray,
                          y_hat: float,
                          y: int) -> np.ndarray:
    # Combine all of the backward gradient steps into a
    # single function
    d_weights = get_gradient_wrt_weights(y_hat, y, flattened)
    d_bias_flat = get_gradient_wrt_bias_flat(y_hat, y)
    d_kernel, d_img_conv = get_gradient_wrt_kernel(
        img_conv, img, y_hat, y, weights, kernel)
    d_bias = get_gradient_wrt_bias(img_conv, d_img_conv)
    return d_kernel, d_bias, d_weights, d_bias_flat

In [17]:
def init_params():
    # Initialise the base state
    # Keep the values small as large values do not work well
    kernel = 0.01*np.random.randn(5, 5)
    bias = 0.01*np.random.randn()
    # Remove the extra 1 dimension from this array
    weights = np.squeeze(0.01*np.random.randn(1,256))
    bias_flat = 0.01*np.random.randn()
    return kernel, bias, weights, bias_flat 

### Synthetic example

In [25]:
# Create arbitraty image and label
img = np.random.randint(1, 255, (32, 32))
y = 0

# Initialise paramaters randomly
kernel, bias, weights, bias_flat  = init_params()

for i in range(5):
    # Perform a forward pass
    img_conv, flattened, y_hat = forward_pass(img, kernel, bias, weights, bias_flat)

    print(y_hat)

    # Perform a back propagation pass to get weight updates
    d_kernel, d_bias, d_weights, d_bias_flat = back_propagation_pass(
        img, img_conv, flattened, weights, kernel, y_hat, y)

    # Use a learning rate and update weights
    alpha = 0.001
    kernel = kernel - alpha * d_kernel
    bias = bias - alpha * d_bias
    weights = weights - alpha * d_weights
    bias_flat = bias_flat - alpha * d_bias_flat

0.5130462625739269
0.45872157021088356
0.3809208747677658
0.3073262590739291
0.20696897709892656
